# Project: Wrangling and Analyze Data

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import os
import tweepy
from tweepy import OAuthHandler
from timeit import default_timer as timer
df_archive = pd.read_csv('twitter-archive-enhanced.csv')

In [ ]:
df_archive.tweet_id.values

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [ ]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

In [ ]:
r = requests.get(url)

In [ ]:
with open(os.path.join("",url.split('/')[-1]), mode = 'wb') as file:
            file.write(r.content)

In [ ]:
df_image = pd.read_csv('image-predictions.tsv',sep='\t')

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [ ]:
consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)

# NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
# df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
# change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
# NOTE TO REVIEWER: this student had mobile verification issues so the following
# Twitter API code was sent to this student from a Udacity instructor
# Tweet IDs for which to gather additional data via Twitter's API
tweet_ids = df_1.tweet_id.values
len(tweet_ids)

# Query Twitter's API for JSON data for each tweet ID in the Twitter archive
count = 0
fails_dict = {}
start = timer()
# Save each tweet's returned JSON as a new line in a .txt file
with open('tweet_json.txt', 'w') as outfile:
    # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
    for tweet_id in tweet_ids:
        count += 1
        print(str(count) + ": " + str(tweet_id))
        try:
            tweet = api.get_status(tweet_id, tweet_mode='extended')
            print("Success")
            json.dump(tweet._json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
            print("Fail")
            fails_dict[tweet_id] = e
            pass
end = timer()
print(end - start)
print(fails_dict)

In [ ]:
dict_temp = []
with open('tweet-json.txt', 'r') as file:
    for line in file:
        status = json.loads(line)
        tweet_id = status['id_str']
        retweet_count = status['retweet_count']
        fav_count = status['favorite_count']
        dict_temp.append({'tweet_id': tweet_id,
                        'retweet_count': retweet_count,
                         'favorite_count': fav_count })
tweets_metrics = pd.DataFrame.from_dict(dict_temp)

In [ ]:
df_archive.info()

In [ ]:
tweets_metrics.info()

In [ ]:
df_image.info()

In [ ]:
df_image.duplicated(subset = 'tweet_id')

In [ ]:
tweets_metrics[tweets_metrics['tweet_id'] == '667152164079423490']

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



# Quality issues
#### Note: most of the issues are from Archive Dataframe, it will be stated if the issue is from another Dataframe
1. Time Stamp have "+0000" at the end
2. Data type is wrong for tweet_id(Archive and Image), TimeStamp, and img_num(Image dataframe)
3. Some records are retweets and replies
4. high amount of NaN values in ('in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id',
'retweeted_status_user_id', 'retweeted_status_timestamp')
5. Source column indicate what is the tweet source, however it's in a link format and its hard to read
6. dog names have incorrect data(such as 'a', 'None')
7. (Image) p1, p2 and p3 columns have a mix of naming breeds with capitalized letters and small letters
8. the rating columns don't have meaning without one another

### Tidiness issues
1. The stages column should be combined into one column (Archive)

2. Merging df_archive with Image and tweet metrics tables


## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [569]:
# Make copies of original pieces of data
df_archive_clean = df_archive.copy()
df_image_clean = df_image.copy()
tweets_metrics_clean = tweets_metrics.copy()

## Quality Cleaning

### Issue #1: Time Stamp have "+0000" at the end

#### Define: Timestamp should only have date and the time, and since every timestamp have "+0000" (which means the time is UCT) we will remove it.

#### Code

In [570]:
df_archive_clean['timestamp'] = df_archive_clean['timestamp'].str[:-6]

#### Test

In [ ]:
df_archive_clean.info()

### Issue #2: Data type is wrong for tweet_id and Time Stamp

#### Define: tweet_id and img_num should be a string not an int since we shouldn't use any calculations on it. Timestamp should be converted to Datetime to be used in date range. I will convert both of them to their appropraite Data types

#### Code

In [592]:
df_archive_clean['timestamp'] = pd.to_datetime(df_archive_clean['timestamp'], format = '%Y-%m-%d %H:%M:%S')
df_archive_clean['tweet_id'] = df_archive_clean['tweet_id'].astype('string')
tweets_metrics_clean['tweet_id'] = tweets_metrics_clean['tweet_id'].astype('string')
df_image_clean['tweet_id'] = df_image_clean['tweet_id'].astype('string')
df_image_clean['img_num'] = df_image_clean['img_num'].astype('string')

#### Test

In [ ]:
df_archive_clean.info()

### Issue #3: Some records are retweets and replies

In [438]:
df_image_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   string 
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   string 
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), object(4), string(2)
memory usage: 152.1+ KB


#### Define: I will drop the retweets and replies.

#### Code

In [572]:
# Dropping the replies using the rows index
index = df_archive_clean[df_archive_clean['in_reply_to_status_id'].notnull()].index
df_archive_clean.drop(index, inplace = True)

In [573]:
# Dropping the retweets using the rows index
index = df_archive_clean[df_archive_clean['retweeted_status_id'].notnull()].index
df_archive_clean.drop(index, inplace = True)

#### Test

In [ ]:
df_archive_clean.info()

### Issue #4: high amount of NaN values in ('in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp')

#### Define: Since we don't need those columns anymore, I will remove them

#### Code

In [574]:
drop_columns = ['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id', 'retweeted_status_user_id', 'retweeted_status_timestamp']
df_archive_clean.drop(drop_columns, axis = 1, inplace = True)

#### Test

In [ ]:
df_archive_clean.info()

### Issue #5: Source column indicate what is the tweet source, however it's in a link format and its hard to read

#### Define: I will extract the exact source using Regex.

#### Code

In [575]:
df_archive_clean['source'] = df_archive_clean['source'].str.extract(r'((?<=\>).+?(?=\<))', expand = True)

#### Test

In [ ]:
df_archive_clean['source'].value_counts()

### Issue #6: dog names have incorrect data(such as 'a', 'None')

#### Define: since we can't retrieve the name, I will replace them with NaN so it can't be biased when we do analysis 

#### Code

In [576]:
incorrect_names = ['a', 'an', 'the', 'None',]
for i in incorrect_names:
    df_archive_clean['name'] = df_archive_clean['name'].replace(i, np.nan)

#### Test

In [ ]:
df_archive_clean[df_archive_clean['name'] == 'a']

In [ ]:
df_archive_clean[df_archive_clean['name'] == 'an']

In [ ]:
df_archive_clean[df_archive_clean['name'] == 'the']

In [ ]:
df_archive_clean[df_archive_clean['name'] == 'None']

### Issue #7: (Image) p1, p2 and p3 columns have a mix of naming breeds with capitalized letters and small letters

#### Define: I will capitalized each column to be consistance

#### Code

In [577]:
df_image_clean['p1'] = df_image_clean['p1'].str.capitalize()
df_image_clean['p2'] = df_image_clean['p2'].str.capitalize()
df_image_clean['p3'] = df_image_clean['p3'].str.capitalize()

#### Test

In [ ]:
df_image_clean.head()

### Issue #8: the rating columns don't have meaning without one another

#### Define: Instead of relying on both columns, I will divide the nominater over the denominator in order to normalize the rating

#### Code

In [578]:
df_archive_clean['rating'] = df_archive_clean['rating_numerator'] / df_archive_clean['rating_denominator']

#### Test

In [ ]:
df_archive_clean.head()

## Tidiness cleaning

### Issue #1: The stages column should be combined into one column (Archive)

#### Define: I will combine the four columns into a single column

#### Code

In [579]:
# replace the 'None' values with a blank
df_archive_clean.doggo.replace('None', '', inplace = True)
df_archive_clean.floofer.replace('None', '', inplace = True)
df_archive_clean.pupper.replace('None', '', inplace = True)
df_archive_clean.puppo.replace('None', '', inplace = True)

In [580]:
# Combine the 4 columns into a single one
df_archive_clean['Stage'] = df_archive_clean.doggo + df_archive_clean.floofer + df_archive_clean.pupper + df_archive_clean.puppo

In [581]:
# replace the blank values with 'Other', which indicate that the stage of the dog is another one that hasn't mentioned
df_archive_clean['Stage'].replace('', 'Other', inplace = True)
# replace the values that have multiple stages with an propriate name
df_archive_clean['Stage'].replace('doggopupper', 'doggo,pupper', inplace = True)
df_archive_clean['Stage'].replace('doggopuppo', 'doggo,puppo', inplace = True)
df_archive_clean['Stage'].replace('doggofloofer', 'doggo,floofer', inplace = True)

In [582]:
# drop the columns which we dont need anymore
dropped_cols = ['doggo', 'floofer', 'pupper', 'puppo']
df_archive_clean.drop(dropped_cols, axis = 1, inplace = True)

#### Test

In [ ]:
df_archive_clean['Stage'].value_counts()

### Issue #1: Merging df_archive with Image and tweet metrics tables

#### Define: merging the dataframes together into a single dataframe

#### Code

In [558]:
tweets_metrics_clean.sample(5)

,tweet_id,retweet_count,favorite_count
791,773670353721753600,1501,5935
2210,668587383441514497,1174,1760
1440,696754882863349760,396,1615
499,813112105746448384,3225,11515
575,800859414831898624,113,778


In [ ]:
747648653817413632

In [561]:
773670353721753600

773670353721753600

In [560]:
df_archive_clean.sample(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,rating,Stage
1006,747648653817413632,2016-06-28 04:31:44,Vine - Make a Scene,This is Keurig. He apparently headbutts other ...,https://vine.co/v/iqIZFtOxEMB,12,10,Keurig,1.2,Other
1331,705591895322394625,2016-03-04 03:13:11,Twitter for iPhone,"""Ma'am, for the last time, I'm not authorized ...",https://twitter.com/dog_rates/status/705591895...,11,10,NaN,1.1,Other
480,815736392542261248,2017-01-02 01:48:06,Twitter for iPhone,This is Akumi. It's his birthday. He received ...,https://twitter.com/dog_rates/status/815736392...,11,10,Akumi,1.1,Other
1159,724046343203856385,2016-04-24 01:24:35,Twitter for iPhone,This is Sarge. Not even he knows what his tong...,https://twitter.com/dog_rates/status/724046343...,10,10,Sarge,1.0,Other
676,789314372632018944,2016-10-21 03:56:25,Twitter for iPhone,HE WAS JUST A LIL SLEEPY FROM BEING SUCH A GOO...,https://twitter.com/sebscat/status/78881832853...,13,10,NaN,1.3,Other


In [601]:
tweets_archive_clean = df_archive_clean.merge(tweets_metrics_clean, how = 'left', on = 'tweet_id')

In [602]:
tweets_archive_clean = tweets_archive_clean.merge(df_image_clean, how = 'left', on = 'tweet_id')

In [603]:
tweets_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2096
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2097 non-null   string        
 1   timestamp           2097 non-null   datetime64[ns]
 2   source              2097 non-null   object        
 3   text                2097 non-null   object        
 4   expanded_urls       2094 non-null   object        
 5   rating_numerator    2097 non-null   int64         
 6   rating_denominator  2097 non-null   int64         
 7   name                1425 non-null   object        
 8   rating              2097 non-null   float64       
 9   Stage               2097 non-null   object        
 10  retweet_count       2097 non-null   int64         
 11  favorite_count      2097 non-null   int64         
 12  jpg_url             1971 non-null   object        
 13  img_num             1971 non-null   string      

In [511]:
tweets_archive_clean.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,rating,Stage,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,892420643555336193,2017-08-01 16:23:56,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,1.3,Other,...,1,Orange,0.097049,False,Bagel,0.085851,False,Banana,0.076110,False
1,892177421306343426,2017-08-01 00:17:27,Twitter for iPhone,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,1.3,Other,...,1,Chihuahua,0.323581,True,Pekinese,0.090647,True,Papillon,0.068957,True
2,891815181378084864,2017-07-31 00:18:03,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,1.2,Other,...,1,Chihuahua,0.716012,True,Malamute,0.078253,True,Kelpie,0.031379,True
3,891689557279858688,2017-07-30 15:58:51,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,1.3,Other,...,1,Paper_towel,0.170278,False,Labrador_retriever,0.168086,True,Spatula,0.040836,False
4,891327558926688256,2017-07-29 16:00:24,Twitter for iPhone,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,1.2,Other,...,2,Basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True


In [513]:
df_image_clean['tweet_id'].dtype

string[python]

In [514]:
tweets_metrics_clean['tweet_id'].dtype

dtype('O')

In [515]:
df_archive_clean['tweet_id'].dtype

string[python]

In [453]:
df_image_clean[df_image_clean['tweet_id'] == '885518971528720385']

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


In [509]:
tweets_archive_clean2[tweets_archive_clean2['source'].isnull()]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,rating,Stage,...,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
94,667550882905632768,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Web_site,0.998258,False,Dishwasher,0.000201,False,Oscilloscope,0.000142,False
95,667550904950915073,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Web_site,0.999335,False,Vizsla,0.000081,True,Collie,0.000069,True
183,669353438988365824,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Teddy,0.379656,False,Pembroke,0.212343,True,Chow,0.096995,True
314,671729906628341761,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Kuvasz,0.431469,True,Samoyed,0.117122,True,White_wolf,0.090067,False
453,674754018082705410,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Seashore,0.352321,False,Promontory,0.131753,False,Wreck,0.095597,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,867072653475098625,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Blenheim_spaniel,0.352946,True,Papillon,0.211766,True,Pekinese,0.112952,True
1992,873697596434513921,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Laptop,0.153718,False,French_bulldog,0.099984,True,Printer,0.077130,False
2006,877611172832227328,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Irish_setter,0.364729,True,Golden_retriever,0.202907,True,Irish_terrier,0.107473,True
2041,885311592912609280,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,Labrador_retriever,0.908703,True,Seat_belt,0.057091,False,Pug,0.011933,True


In [517]:
tweets_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2097 entries, 0 to 2096
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2097 non-null   object        
 1   timestamp           2097 non-null   datetime64[ns]
 2   source              2097 non-null   object        
 3   text                2097 non-null   object        
 4   expanded_urls       2094 non-null   object        
 5   rating_numerator    2097 non-null   int64         
 6   rating_denominator  2097 non-null   int64         
 7   name                1425 non-null   object        
 8   rating              2097 non-null   float64       
 9   Stage               2097 non-null   object        
 10  retweet_count       2097 non-null   int64         
 11  favorite_count      2097 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(6)
memory usage: 213.0+ KB


## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization